# Install

In [1]:
!pip install -q dwave-ocean-sdk

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from google.colab import files
import pickle

In [3]:
from collections import defaultdict
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dwave.inspector as inspector
from dimod import ConstrainedQuadraticModel, CQM, SampleSet
from dwave.system import LeapHybridCQMSampler
from dimod.vartypes import Vartype
from dimod import Binary, quicksum

In [4]:
from google.colab import userdata

# Import Alpha, Beta and Theta data

In [206]:
!rm *

rm: cannot remove 'sample_data': Is a directory


## Alpha data

In [207]:
alpha_data = files.upload()
alpha_file_name = list(alpha_data.keys())[0]
alpha_file_name

Saving Vision_400_alpha.txt to Vision_400_alpha.txt


'Vision_400_alpha.txt'

In [208]:
alpha = dict()
with open(alpha_file_name) as f:
    lines = f.readlines()
    for line in lines:
      a,b,c = line.replace('\n','').split(' ')
      a = int(a)
      b = int(b)
      c = int(c)
      alpha[(a-1,b-1)] = c
#alpha

## Beta data

In [209]:
beta_data = files.upload()
beta_file_name = list(beta_data.keys())[0]
beta_file_name

Saving Vision_400_beta.txt to Vision_400_beta.txt


'Vision_400_beta.txt'

In [210]:
n_students = 0
with open(beta_file_name) as f:
    lines = f.readlines()
    first_line = lines[0].split(' ')
    #print(first_line)
    n_features = len(first_line)
    #print(n_features)
    beta = [[] for x in range(n_features)]
    for line in lines:
      n_students +=1
      line = line.replace('\n','').split(' ')
      for i in range(n_features):
        beta[i].append(int(line[i]))

In [211]:
for i in range(n_students):
  for j in range(n_students):
    if i==j:
      alpha[(i,j)] = 0

## Theta data

In [212]:
theta_data = files.upload()
theta_file_name = list(theta_data.keys())[0]
theta_file_name

Saving Vision_400_theta.txt to Vision_400_theta.txt


'Vision_400_theta.txt'

In [213]:
theta = []
with open(theta_file_name) as f:
    lines = f.readlines()
    for line in lines:
      theta.append(int(line.replace('\n','')))
theta

[6, 6, 9, 11]

In [214]:
beta_max = [theta[x] for x in range(n_features)]
tao_min = theta[-2]
tao_max = theta[-1]

## Check problem inputs

In [215]:
n_groups = 40

In [216]:
print(f'Arrange classroom of {n_students} students, into {n_groups} groups of min group \
size: {tao_min} and max group size: {tao_max}')

Arrange classroom of 400 students, into 40 groups of min group size: 9 and max group size: 11


# DWAVE Initializing

In [217]:
endpoint = 'https://cloud.dwavesys.com/sapi'
token = userdata.get('dwave_leap')

In [218]:
N,C = range(n_students),range(n_groups)

# Creating Model

In [219]:
cqm = ConstrainedQuadraticModel()

## Creating the variables

In [220]:
x = {(i, c): Binary(f'x{i}_{c}') for i in range(n_students) for c in range(n_groups)}

## η variables

In [221]:
eta_qbutis = int(np.floor(np.log2(n_students**2)) + 1)
eta_qbutis

18

In [222]:
eta_bin_coeffs = [2**i for i in range(eta_qbutis)]
eta_bin_coeffs

[1,
 2,
 4,
 8,
 16,
 32,
 64,
 128,
 256,
 512,
 1024,
 2048,
 4096,
 8192,
 16384,
 32768,
 65536,
 131072]

In [223]:
E = range(eta_qbutis)

In [224]:
eta = {i: Binary(f'eta{i}') for i in E}

## Creating the objective function

In [225]:
objective = -1*quicksum(eta[i]*eta_bin_coeffs[i] for i in E)
cqm.set_objective(objective)

## Creating the constraints

In [226]:
# one student per group

for i in N:
  cqm.add_constraint( quicksum(x[i,c] for c in C) == 1 )

In [227]:
# minimum students per group >= tao_min
# Maximum students per group <= tao_max

for c in C:
  cqm.add_constraint( quicksum(x[i,c] for i in N) >= tao_min )
  cqm.add_constraint( quicksum(x[i,c] for i in N) <= tao_max )

In [228]:
# Beta homegeinity constraints
group_combs = [(c,cp) for c in C for cp in C if c!=cp]
#group_combs

In [229]:
# eta constraints

for c in C:
  cqm.add_constraint(  quicksum(x[i,c]*x[j,c]*alpha.get((i,j),0) for i in N for j in N) - quicksum(eta[e]*eta_bin_coeffs[e] for e in E) >= 0  )

In [230]:
for c1,c2 in group_combs:
  for f in range(n_features):
    bf = beta[f]
    B = beta_max[f]
    cqm.add_constraint( quicksum(x[i,c1]*bf[i] for i in N) - quicksum(x[i,c2]*bf[i] for i in N) <= B )

## Run the model

In [231]:
counter = 1
while True:
  try:
    cqm_sampler = LeapHybridCQMSampler(endpoint=endpoint, token=token)
  except:
    if counter <= 5:
      print(f"{counter} -  Problem finding embedding trying it once again...")
      counter += 1
      continue
    else:
      raise Exception(f"Imposible to find an embedding after {counter} tries")
  break

In [232]:
sampleset = cqm_sampler.sample_cqm(cqm,label = f'CGFP_MinMax_{n_students}')

In [233]:
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)

In [234]:
not_feasible_sampleset = sampleset.filter(lambda row: not row.is_feasible)

In [235]:
best = feasible_sampleset.first

In [236]:
best_solution = best.sample

In [237]:
Z = -1*best.energy
Z

2.0

# Verify best solution constraints

In [238]:
def verify_constraints(sample,print_all = False,verbose = False):

  violated = 0

  for i in N:
    just_one = [sample[f'x{i}_{c}'] for c in C]
    if sum(just_one) != 1:
      violated += 1
      if verbose:
        print(f'Individual {i} with {sum(just_one)} assignments')
    if verbose:
      if print_all:
        print(f'Individual {i} with {sum(just_one)} assignments')

  GROUPS = [ []  for g in C]
  for var in sample:
    if 'eta' in var:
      continue
    if sample[var]==1:
      st,g = var[1:].split('_')
      st = int(st)
      g = int(g)
      GROUPS[g].append(st)
      # print(f'student_{st} belongs to group {g}')

  for i,g in enumerate(GROUPS):
    spg = len(g)
    valid = spg >= tao_min and spg <= tao_max
    if not valid:
      violated += 1
      if verbose:
        print(f'* number of members of group_{i}: ({tao_min} <= {len(g)} <= {tao_max}), valid = {valid}')
    if verbose:
      if print_all:
        print(f'* number of members of group_{i}: ({tao_min} <= {len(g)} <= {tao_max}), valid = {valid}')
  for c1,c2 in group_combs:

    G1 = GROUPS[c1]
    G2 = GROUPS[c2]
    for f in range(n_features):
      bf = beta[f]
      B = beta_max[f]
      val1 = sum([bf[x] for x in G1])
      val2 = sum([bf[x] for x in G2])
      subs = val1 - val2
      valid = subs >= -B and subs <= B

      if not valid:
        violated += 1
        if verbose:
          print(f'(g{c1},g{c2}) feature = {f},| {-B} <= {subs} <= {B}| valid = {valid} ')
      if verbose:
        if print_all:
          print(f'(g{c1},g{c2}) feature = {f},| {-B} <= {subs} <= {B}| valid = {valid} ')
  return violated

In [239]:
GROUPS = [ []  for g in C]
for var in best_solution:
  if 'eta' in var:
    continue
  if best_solution[var]==1:
    st,g = var[1:].split('_')
    st = int(st)
    g = int(g)
    GROUPS[g].append(st)
    print(f'student_{st} belongs to group {g}')

student_0 belongs to group 18
student_100 belongs to group 3
student_101 belongs to group 26
student_102 belongs to group 26
student_103 belongs to group 13
student_104 belongs to group 19
student_105 belongs to group 14
student_106 belongs to group 17
student_107 belongs to group 36
student_108 belongs to group 32
student_109 belongs to group 12
student_10 belongs to group 36
student_110 belongs to group 3
student_111 belongs to group 15
student_112 belongs to group 0
student_113 belongs to group 37
student_114 belongs to group 11
student_115 belongs to group 20
student_116 belongs to group 2
student_117 belongs to group 23
student_118 belongs to group 5
student_119 belongs to group 5
student_11 belongs to group 8
student_120 belongs to group 11
student_121 belongs to group 11
student_122 belongs to group 19
student_123 belongs to group 28
student_124 belongs to group 34
student_125 belongs to group 15
student_126 belongs to group 9
student_127 belongs to group 18
student_128 belongs 

In [240]:
for i,g in enumerate(GROUPS):
  spg = len(g)
  valid = spg >= tao_min and spg <= tao_max
  print(f'* number of members of group_{i}: ({tao_min} <= {len(g)} <= {tao_max}), valid = {valid}')

* number of members of group_0: (9 <= 11 <= 11), valid = True
* number of members of group_1: (9 <= 9 <= 11), valid = True
* number of members of group_2: (9 <= 10 <= 11), valid = True
* number of members of group_3: (9 <= 9 <= 11), valid = True
* number of members of group_4: (9 <= 9 <= 11), valid = True
* number of members of group_5: (9 <= 10 <= 11), valid = True
* number of members of group_6: (9 <= 10 <= 11), valid = True
* number of members of group_7: (9 <= 10 <= 11), valid = True
* number of members of group_8: (9 <= 10 <= 11), valid = True
* number of members of group_9: (9 <= 9 <= 11), valid = True
* number of members of group_10: (9 <= 10 <= 11), valid = True
* number of members of group_11: (9 <= 11 <= 11), valid = True
* number of members of group_12: (9 <= 9 <= 11), valid = True
* number of members of group_13: (9 <= 10 <= 11), valid = True
* number of members of group_14: (9 <= 11 <= 11), valid = True
* number of members of group_15: (9 <= 9 <= 11), valid = True
* number

In [241]:
for c1,c2 in group_combs:
  #print((c1,c2),GROUPS[c1],GROUPS[c2])
  G1 = GROUPS[c1]
  G2 = GROUPS[c2]
  for f in range(n_features):
    bf = beta[f]
    B = beta_max[f]
    #print(bf)
    val1 = sum([bf[x] for x in G1])
    val2 = sum([bf[x] for x in G2])
    subs = val1 - val2
    valid = subs >= -B and subs <= B

    print(f'(g{c1},g{c2}) feature = {f},| {-B} <= {subs} <= {B}| valid = {valid} ')

(g0,g1) feature = 0,| -6 <= -2 <= 6| valid = True 
(g0,g1) feature = 1,| -6 <= 4 <= 6| valid = True 
(g0,g2) feature = 0,| -6 <= -3 <= 6| valid = True 
(g0,g2) feature = 1,| -6 <= -1 <= 6| valid = True 
(g0,g3) feature = 0,| -6 <= -3 <= 6| valid = True 
(g0,g3) feature = 1,| -6 <= 0 <= 6| valid = True 
(g0,g4) feature = 0,| -6 <= -4 <= 6| valid = True 
(g0,g4) feature = 1,| -6 <= -1 <= 6| valid = True 
(g0,g5) feature = 0,| -6 <= -3 <= 6| valid = True 
(g0,g5) feature = 1,| -6 <= -2 <= 6| valid = True 
(g0,g6) feature = 0,| -6 <= -2 <= 6| valid = True 
(g0,g6) feature = 1,| -6 <= 1 <= 6| valid = True 
(g0,g7) feature = 0,| -6 <= -5 <= 6| valid = True 
(g0,g7) feature = 1,| -6 <= 1 <= 6| valid = True 
(g0,g8) feature = 0,| -6 <= -2 <= 6| valid = True 
(g0,g8) feature = 1,| -6 <= -1 <= 6| valid = True 
(g0,g9) feature = 0,| -6 <= -2 <= 6| valid = True 
(g0,g9) feature = 1,| -6 <= 1 <= 6| valid = True 
(g0,g10) feature = 0,| -6 <= -3 <= 6| valid = True 
(g0,g10) feature = 1,| -6 <= -1 <= 

In [242]:
z_obj = 0
groups_happiness = []
for group in GROUPS:
  happy = 0
  for s1 in group:
    for s2 in group:
      if s1 != s2:
        happy += alpha.get((s1,s2),0)
        z_obj += alpha.get((s1,s2),0)
  groups_happiness.append(happy)

In [243]:
for i,g in enumerate(GROUPS):
  print(f'group {i}, hapiness = {groups_happiness[i]}')
print(f'Total Happiness = {z_obj}')

group 0, hapiness = 13
group 1, hapiness = 10
group 2, hapiness = 5
group 3, hapiness = 9
group 4, hapiness = 5
group 5, hapiness = 2
group 6, hapiness = 7
group 7, hapiness = 10
group 8, hapiness = 2
group 9, hapiness = 2
group 10, hapiness = 3
group 11, hapiness = 14
group 12, hapiness = 7
group 13, hapiness = 6
group 14, hapiness = 8
group 15, hapiness = 4
group 16, hapiness = 3
group 17, hapiness = 9
group 18, hapiness = 3
group 19, hapiness = 7
group 20, hapiness = 12
group 21, hapiness = 11
group 22, hapiness = 4
group 23, hapiness = 3
group 24, hapiness = 9
group 25, hapiness = 4
group 26, hapiness = 9
group 27, hapiness = 3
group 28, hapiness = 8
group 29, hapiness = 6
group 30, hapiness = 16
group 31, hapiness = 4
group 32, hapiness = 5
group 33, hapiness = 3
group 34, hapiness = 16
group 35, hapiness = 10
group 36, hapiness = 11
group 37, hapiness = 15
group 38, hapiness = 3
group 39, hapiness = 7
Total Happiness = 288


# Verify all not feasible constraints

In [244]:
for sample in not_feasible_sampleset:
  print(verify_constraints(sample))
  print("-"*120)

1378
------------------------------------------------------------------------------------------------------------------------
34
------------------------------------------------------------------------------------------------------------------------


# Extract solution txt

In [245]:
x_dict = dict()
for var in best_solution:
  if 'eta' in var:
    continue
  if best_solution[var]==1:
    st,g = var[1:].split('_')
    st = int(st) + 1
    g = int(g) + 1
    x_dict[st] = g
x_dict = sorted(x_dict.items(), key=lambda x:x[0])

In [246]:
groups = [[] for x in range(n_groups)]
zetas = [0 for x in range(n_groups)]
for x,group in x_dict:
  groups[group-1].append(x)


for i,G in enumerate(groups):
  happy = 0
  for s1 in G:
    for s2 in G:
      if s1 != s2:
        happy += alpha.get((s1-1,s2-1),0)
  print(f"group {i}, Z = {happy}")

group 0, Z = 13
group 1, Z = 10
group 2, Z = 5
group 3, Z = 9
group 4, Z = 5
group 5, Z = 2
group 6, Z = 7
group 7, Z = 10
group 8, Z = 2
group 9, Z = 2
group 10, Z = 3
group 11, Z = 14
group 12, Z = 7
group 13, Z = 6
group 14, Z = 8
group 15, Z = 4
group 16, Z = 3
group 17, Z = 9
group 18, Z = 3
group 19, Z = 7
group 20, Z = 12
group 21, Z = 11
group 22, Z = 4
group 23, Z = 3
group 24, Z = 9
group 25, Z = 4
group 26, Z = 9
group 27, Z = 3
group 28, Z = 8
group 29, Z = 6
group 30, Z = 16
group 31, Z = 4
group 32, Z = 5
group 33, Z = 3
group 34, Z = 16
group 35, Z = 10
group 36, Z = 11
group 37, Z = 15
group 38, Z = 3
group 39, Z = 7


# Save best solution txt

In [247]:
#one solution only

# f = open("x_min_max_100.txt", "w")
# for i,j in x_dict:
#   f.write(f"{i} {j}\n")
# f.write(f"{Z}")
# f.close()

# Extract Multiple Solution

In [248]:
def generate_all_solutions_dict(sampleset):

  solutions = []

  for sample in sampleset:
    sample = dict(sample)
    x_dict = dict()
    for var in sample:
      if 'eta' in var:
        continue
      if sample[var]==1:
        st,g = var[1:].split('_')
        st = int(st) + 1
        g = int(g) + 1
        x_dict[st] = g
    x_dict = sorted(x_dict.items(), key=lambda x:x[0])
    solutions.append(x_dict)
  return solutions

## Save multiple feasible solutions and entire sampleset

In [249]:
all_feasible_x = generate_all_solutions_dict(feasible_sampleset)

In [250]:
# multiple solutions

f = open(f"Vision_{n_students}_x_min_max_all_feasible.txt", "w")
f.write(f"{len(all_feasible_x)}\n")
for x_d in all_feasible_x:
  for i,j in x_d:
    f.write(f"{i} {j}\n")
  # f.write("---\n")
f.close()

In [251]:
with open(f"Vision_{n_students}_x_min_max_feasible_sampleset.pkl", 'wb') as f:
  pickle.dump(feasible_sampleset, f)

In [252]:
with open(f"Vision_{n_students}_x_min_max_all_sampleset.pkl", 'wb') as f:
  pickle.dump(sampleset, f)

## Save not feasible solutions

In [253]:
all_not_feasible_x = generate_all_solutions_dict(not_feasible_sampleset)

In [254]:
# # multiple solutions

# f = open(f"x_min_max_{n_students}_multiple_not_feasible.txt", "w")
# f.write(f"{len(all_not_feasible_x)}\n")
# for x_d in all_not_feasible_x:
#   for i,j in x_d:
#     f.write(f"{i} {j}\n")
#   # f.write("---\n")
# f.close()

In [255]:
# # save it as a pickle file

# with open(f"x_min_max_{n_students}_multiple_not_feasible.pkl", 'wb') as f:
#   pickle.dump(all_not_feasible_x, f)